In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import time
import warnings
warnings.filterwarnings('ignore')
import json
!pip install nltk
!pip install dask
import dask as dd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

In [7]:
instrument = pd.read_csv('Musical_Instruments_data (1).csv')
instrument.head(2)

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5.0,90,False,"08 9, 2004",AXHY24HWOF184,0470536454,{'Format:': ' Paperback'},Bendy,Crocheting for Dummies by Karen Manthey & Susa...,Terrific Book for Learning the Art of Crochet,1092009600,NaN
1,4.0,2,True,"04 6, 2017",A29OWR79AM796H,0470536454,{'Format:': ' Hardcover'},Amazon Customer,Very helpful...,Four Stars,1491436800,NaN


In [8]:
instrument.shape

(1512530, 12)

In [9]:
instrument.drop(columns=['vote','image','style'],inplace=True)

In [10]:
instrument.isnull().sum()

overall             0
verified            0
reviewTime          0
reviewerID          0
asin                0
reviewerName      212
reviewText        855
summary           391
unixReviewTime      0
dtype: int64

In [11]:
instrument.dropna(inplace=True)

In [12]:
instrument.shape

(1511106, 9)

In [13]:
instrument.isnull().sum()

overall           0
verified          0
reviewTime        0
reviewerID        0
asin              0
reviewerName      0
reviewText        0
summary           0
unixReviewTime    0
dtype: int64

In [14]:
instrument.head(2)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime
0,5.0,False,"08 9, 2004",AXHY24HWOF184,0470536454,Bendy,Crocheting for Dummies by Karen Manthey & Susa...,Terrific Book for Learning the Art of Crochet,1092009600
1,4.0,True,"04 6, 2017",A29OWR79AM796H,0470536454,Amazon Customer,Very helpful...,Four Stars,1491436800


In [15]:
instrument_meta=pd.read_csv('Musical_Instruments_meta.csv')

In [16]:
instrument_meta.head(2)

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"['Musical Instruments', 'Drums & Percussion', ...",NaN,"[""Cricket Rubbing the spine with the wooden st...",NaN,Wooden Percussion 2 Piece Set of 3 Inch Cricke...,"['B00NP8GYVS', 'B00NP80XMO', 'B00NP8M098']",NaN,WADSUWAN SHOP,"['Wood percussion', 'Owl whistle*', 'Includes ...","['>#141,729 in Musical Instruments (See Top 10...",[],Musical Instruments,NaN,"December 2, 2013",NaN,0000989983,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,NaN
1,"['Musical Instruments', 'Drums & Percussion', ...",NaN,"[""Frog - Rubbing its spine with the wooden sti...",NaN,"Wooden Percussion 3 Piece Set Frog, Cricket an...","['B00NP8GYVS', 'B00NP80XMO', 'B01MY48HK5', 'B0...",NaN,WADSUWAN SHOP,"['Wood percussion', 'Small 3 inches', 'Creates...","['>#1,622 in Musical Instruments (See Top 100 ...",[],Musical Instruments,NaN,"December 2, 2013",$0.91,0000098906,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,NaN


In [17]:
instrument_meta.shape

(120310, 19)

In [18]:
instrument_meta.isnull().sum()

category                0
tech1              119061
description             0
fit                120307
title                   0
also_buy                0
tech2              120295
brand                5460
feature                 0
rank                    0
also_view               0
main_cat              642
similar_item        74859
date                 2421
price               58393
asin                    0
imageURL                0
imageURLHighRes         0
details               116
dtype: int64

In [19]:
instrument_meta.drop(columns=['tech1','fit','tech2','similar_item',
                         'date','imageURL','imageURLHighRes','main_cat','feature'],inplace=True)

In [20]:
instrument_meta.head(2)

,category,description,title,also_buy,brand,rank,also_view,price,asin,details
0,"['Musical Instruments', 'Drums & Percussion', ...","[""Cricket Rubbing the spine with the wooden st...",Wooden Percussion 2 Piece Set of 3 Inch Cricke...,"['B00NP8GYVS', 'B00NP80XMO', 'B00NP8M098']",WADSUWAN SHOP,"['>#141,729 in Musical Instruments (See Top 10...",[],NaN,0000989983,NaN
1,"['Musical Instruments', 'Drums & Percussion', ...","[""Frog - Rubbing its spine with the wooden sti...","Wooden Percussion 3 Piece Set Frog, Cricket an...","['B00NP8GYVS', 'B00NP80XMO', 'B01MY48HK5', 'B0...",WADSUWAN SHOP,"['>#1,622 in Musical Instruments (See Top 100 ...",[],$0.91,0000098906,NaN


In [ ]:
instrument_meta.shape

In [21]:
instrument_meta.isnull().sum()

category           0
description        0
title              0
also_buy           0
brand           5460
rank               0
also_view          0
price          58393
asin               0
details          116
dtype: int64

In [22]:
instrument_meta.dropna(subset=['title','brand','details'],inplace=True)

In [23]:
def cleaning(text):
    
    text = re.sub("[^0-9\-]+","",text) # remove all non letters & non-spaces except hyphen $ space

    return text

In [24]:
instrument_meta['rank'] = instrument_meta['rank'].apply(str)

In [25]:
instrument_meta['rank'] = instrument_meta['rank'].apply(cleaning)

In [26]:
instrument_meta.head(2)

,category,description,title,also_buy,brand,rank,also_view,price,asin,details
116,"['Musical Instruments', 'Live Sound & Stage', ...",['New Waterproof Aluminum encased rigid LED li...,19.6 Inches Waterproof Rigid LED Strip Light B...,[],CBconcept,48850010011404143337,[],NaN,7109113205,{}
117,[],"['10 of his greatest, including: Hard To Say ?...",Hal Leonard Dan Fogelberg Greatest Hits,[],Hal Leonard,80007100,[],$16.99,7399956812,{}


In [27]:
instrument_meta.isnull().sum()

category           0
description        0
title              0
also_buy           0
brand              0
rank               0
also_view          0
price          53241
asin               0
details            0
dtype: int64

In [28]:
instrument_meta.dtypes

category       object
description    object
title          object
also_buy       object
brand          object
rank           object
also_view      object
price          object
asin           object
details        object
dtype: object

In [29]:
instrument_meta['rank']=pd.to_numeric(instrument_meta['rank'],errors='coerce')

In [30]:
instrument_meta.isnull().sum()

category           0
description        0
title              0
also_buy           0
brand              0
rank            6174
also_view          0
price          53241
asin               0
details            0
dtype: int64

In [31]:
instrument_meta.dropna(subset=['rank','price'],inplace=True)

In [32]:
instrument_meta['price'] = instrument_meta['price'].apply(lambda x:x.replace('$',''))

In [33]:
instrument_meta['price']=pd.to_numeric(instrument_meta['price'],errors='coerce')

In [34]:
instrument_meta.dropna(inplace=True)

In [35]:
instrument_meta.dtypes

category        object
description     object
title           object
also_buy        object
brand           object
rank           float64
also_view       object
price          float64
asin            object
details         object
dtype: object

In [36]:
def clean(text):
    text = re.sub('[^0-9A-Za-z\-]+',' ',text) #remove all non-letters & non-spaces except hyphen & space
    text = re.sub('(?<!\w)\d+','',text) #if number is written after a word remove it
    text = re.sub('-(?!\w)','',text) #remove hyphen except for words which need hyphen eg. tik-tok
    text = " ".join(text.split()) #remove multiple cases
    text = text.lower()
    return text

In [37]:
instrument['text_process'] = instrument['reviewText'].apply(clean)

In [38]:
stop_words = stopwords.words('english')
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [39]:
lemmatizer = WordNetLemmatizer()

instrument['text_process'] = instrument['text_process'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split() if word not in stop_words]))

In [40]:
instrument_data=pd.merge(instrument,instrument_meta,on='asin')

In [41]:
instrument_data.head(2)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,text_process,category,description,title,also_buy,brand,rank,also_view,price,details
0,4.0,True,"11 8, 2011",A3RZ644OLW1X4K,9792372326,Melanie Arthur,I initially purchased the Mackie SRM450v2 beca...,K10 vs Mackie SRM450v2 vs Klipsch bookshelf,1320710400,initially purchased mackie srm450v2 fan mackie...,"['Musical Instruments', 'Live Sound &amp; Stag...","['QSC K10 2-Way Powered Speaker - 1000 Watts, ...","QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",[],QSC,6.669710e+15,"['B06Y4BT184', 'B00CE0VLXO', 'B00NIE80A2', 'B0...",125.99,{}
1,5.0,False,"10 18, 2011",A2Z89YMZZJWBHS,9792372326,Sunn Kram,I have been using QSC K10s for close to a coup...,QSC K10- Best In Class,1318896000,using qsc k10s close couple year solo performe...,"['Musical Instruments', 'Live Sound &amp; Stag...","['QSC K10 2-Way Powered Speaker - 1000 Watts, ...","QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",[],QSC,6.669710e+15,"['B06Y4BT184', 'B00CE0VLXO', 'B00NIE80A2', 'B0...",125.99,{}


In [42]:
instrument_data.shape

(1259994, 19)

In [43]:
k=range(len(instrument_data))
ki=[]
for i in k:
    m=instrument_data['category'][i][25:45]
    ki.append(m)

In [44]:
instrument_data['instrument_category']=ki

In [45]:
instrument_data['instrument_category'].value_counts()

Instrument Accessori    611517
Microphones & Access    123715
Live Sound & Stage',    109027
Studio Recording Equ     70574
Drums & Percussion',     52862
Amplifiers & Effects     48236
                         37472
Electronic Music, DJ     35524
Wind & Woodwind Inst     29199
Ukuleles, Mandolins      27673
Band & Orchestra', '     26754
Guitars', 'Electric      15254
Keyboards & MIDI', '     14451
Microphones &amp; Ac     12384
Live Sound &amp; Sta     10726
Drums &amp; Percussi      8295
Wind &amp; Woodwind       5140
Guitars', 'Acoustic       4339
Band &amp; Orchestra      3435
Amplifiers &amp; Eff      3122
Drums & Percussion']      3094
Guitars']                 2391
Bass Guitars', 'Elec      2197
Stringed Instruments      1727
Keyboards &amp; MIDI       380
Live Sound & Stage']       284
Keyboards & MIDI']         169
Band & Orchestra']          18
Bass Guitars', 'Begi        18
Bass Guitars']              17
Name: instrument_category, dtype: int64

In [46]:
def cat(existing,replace):
    instrument_data['instrument_category'] = instrument_data['instrument_category'].apply(lambda x:x.replace(existing,replace))
    return instrument_data

In [47]:
instrument_data=cat('Instrument Accessori','Instrument Accessories')
instrument_data=cat('Microphones & Access','Microphones & Accessories')
instrument_data=cat("Live Sound & Stage',",'Live Sound & Stage')
instrument_data=cat('Studio Recording Equ','Studio Recording Equipment')
instrument_data=cat("Drums & Percussion',",'Drums & Percussion')
instrument_data=cat('Electronic Music, DJ','Electronic Music')
instrument_data=cat('Wind & Woodwind Inst','Wind & Woodwind Instrument')
instrument_data=cat('Ukuleles, Mandolins ','Ukuleles')
instrument_data=cat("Band & Orchestra', '",'Band & Orchestra')
instrument_data=cat('Amplifiers &amp; Eff','Amplifiers &amp')
instrument_data=cat("Guitars', 'Electric ",'Guitars')
instrument_data=cat("Keyboards & MIDI', '",'Keyboards')
instrument_data=cat('Microphones &amp; Ac','Microphones &amp')
instrument_data=cat('Live Sound &amp; Sta','Live Sound &amp')
instrument_data=cat('Drums &amp; Percussi','Drums &amp')
instrument_data=cat('Wind &amp; Woodwind ','Wind &amp')
instrument_data=cat("Guitars', 'Acoustic ",'Guitars')
instrument_data=cat('Band &amp; Orchestra','Band &amp')
instrument_data=cat("Drums & Percussion']",'Drums & Percussion')
instrument_data=cat("Guitars']",'Guitars')
instrument_data=cat("Bass Guitars', 'Elec",'Guitars')
instrument_data=cat('Keyboards &amp; MIDI','Keyboards &amp')
instrument_data=cat("Live Sound & Stage']",'Live Sound & Stage')
instrument_data=cat("Keyboards & MIDI']",'Keyboards')
instrument_data=cat("Band & Orchestra']",'Band & Orchestra')
instrument_data=cat("Bass Guitars', 'Begi",'Guitars')
instrument_data=cat("Bass Guitars']",'Guitars')
instrument_data=cat("Bass Guitars",'Guitars')

In [48]:
instrument_data['instrument_category'].value_counts()

Instrument Accessories        611517
Microphones & Accessories     123715
Live Sound & Stage            109311
Studio Recording Equipment     70574
Drums & Percussion             55956
Amplifiers & Effects           48236
                               37472
Electronic Music               35524
Wind & Woodwind Instrument     29199
Ukuleles                       27673
Band & Orchestra               26772
Guitars                        24216
Keyboards                      14620
Microphones &amp               12384
Live Sound &amp                10726
Drums &amp                      8295
Wind &amp                       5140
Band &amp                       3435
Amplifiers &amp                 3122
Stringed Instruments            1727
Keyboards &amp                   380
Name: instrument_category, dtype: int64

In [49]:
instrument_data.isnull().sum()

overall                0
verified               0
reviewTime             0
reviewerID             0
asin                   0
reviewerName           0
reviewText             0
summary                0
unixReviewTime         0
text_process           0
category               0
description            0
title                  0
also_buy               0
brand                  0
rank                   0
also_view              0
price                  0
details                0
instrument_category    0
dtype: int64

In [50]:
instrument_data.dtypes

overall                float64
verified                  bool
reviewTime              object
reviewerID              object
asin                    object
reviewerName            object
reviewText              object
summary                 object
unixReviewTime           int64
text_process            object
category                object
description             object
title                   object
also_buy                object
brand                   object
rank                   float64
also_view               object
price                  float64
details                 object
instrument_category     object
dtype: object

In [51]:
def vader_sentiment_analyzer(df):
    d=time.time()
    sentiments = SentimentIntensityAnalyzer()
    df["compound"] = [sentiments.polarity_scores(i)["compound"] for i in df["text_process"]]
    df['vader_sentiment']=range(len(df))
    df['vader_sentiment'].loc[df['compound'] >=0.5]='Positive'
    df['vader_sentiment'].loc[(df['compound'] >= 0) & (df['compound'] < 0.5)]='Neutral'
    df['vader_sentiment'].loc[(df['compound'] < 0) & (df['compound'] > -0.5)]='Conflict'
    df['vader_sentiment'].loc[df['compound'] <=-0.5]='Negative'
    e=time.time()
    f=d-e
    return(df,f)

In [52]:
instrument_data,vader1time=vader_sentiment_analyzer(instrument_data)

In [56]:
instrument_data

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,text_process,...,title,also_buy,brand,rank,also_view,price,details,instrument_category,compound,vader_sentiment
0,4.0,True,"11 8, 2011",A3RZ644OLW1X4K,9792372326,Melanie Arthur,I initially purchased the Mackie SRM450v2 beca...,K10 vs Mackie SRM450v2 vs Klipsch bookshelf,1320710400,initially purchased mackie srm450v2 fan mackie...,...,"QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",[],QSC,6.669710e+15,"['B06Y4BT184', 'B00CE0VLXO', 'B00NIE80A2', 'B0...",125.99,{},Live Sound &amp,0.9862,Positive
1,5.0,False,"10 18, 2011",A2Z89YMZZJWBHS,9792372326,Sunn Kram,I have been using QSC K10s for close to a coup...,QSC K10- Best In Class,1318896000,using qsc k10s close couple year solo performe...,...,"QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",[],QSC,6.669710e+15,"['B06Y4BT184', 'B00CE0VLXO', 'B00NIE80A2', 'B0...",125.99,{},Live Sound &amp,0.9981,Positive
2,5.0,False,"03 6, 2011",A2GWSCGKIKHI1Z,9792372326,D. Chan,I like this speaker. I bought a pair of K10s b...,Lots of Power,1299369600,like speaker bought pair k10s wanted stereo im...,...,"QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",[],QSC,6.669710e+15,"['B06Y4BT184', 'B00CE0VLXO', 'B00NIE80A2', 'B0...",125.99,{},Live Sound &amp,0.9743,Positive
3,5.0,True,"07 6, 2010",A3CB6OWKSNLZI8,9792372326,jose ricardo dario de pavia Iturralde,Well let me tell you i was an mackie srm 450 u...,The best powered speaker so far!,1278374400,well let tell mackie srm user good powered spe...,...,"QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",[],QSC,6.669710e+15,"['B06Y4BT184', 'B00CE0VLXO', 'B00NIE80A2', 'B0...",125.99,{},Live Sound &amp,0.8481,Positive
4,5.0,True,"09 21, 2016",A27HBIWF77N3H2,9792372326,Brian Hames,One of the best speakers on the market. This t...,Five Stars,1474416000,one best speaker market thing thump,...,"QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",[],QSC,6.669710e+15,"['B06Y4BT184', 'B00CE0VLXO', 'B00NIE80A2', 'B0...",125.99,{},Live Sound &amp,0.6369,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259989,4.0,True,"02 16, 2018",A2O9BRUKIDZYNO,B01HJDOF2Y,Matt Tucker,"Good price for what you get, but out of the 10...",Decent value,1518739200,good price get least switch work yet one used ...,...,10 pcs 3pdt Stomp Footswitch incl PCB incl met...,[],3pdt,2.333410e+11,[],7.99,{},Amplifiers & Effects,0.7096,Positive
1259990,3.0,True,"01 4, 2018",AM3N1PA9QL6LQ,B01HJDOF2Y,tonelab,Love the switches. Missing some metal washers...,May not contain all the hardware (metal washe...,1515024000,love switch missing metal washer though one se...,...,10 pcs 3pdt Stomp Footswitch incl PCB incl met...,[],3pdt,2.333410e+11,[],7.99,{},Amplifiers & Effects,0.4588,Neutral
1259991,5.0,True,"12 31, 2017",AZ6NGFDUGTO8I,B01HJDOF2Y,Amazon Customer,Added PCB is useful and helps keep things orga...,Added PCB is useful and helps keep things orga...,1514678400,added pcb useful help keep thing organized pac...,...,10 pcs 3pdt Stomp Footswitch incl PCB incl met...,[],3pdt,2.333410e+11,[],7.99,{},Amplifiers & Effects,0.8232,Positive
1259992,5.0,True,"01 7, 2017",A25PD56PHCIUQJ,B01HJDOF2Y,Amazon Customer,"Packaging is DOPE. I love the clear tubes, and...",Awesome Packaging and THEY DO come with PCBs,1483747200,packaging dope love clear tube pdts come mount...,...,10 pcs 3pdt Stomp Footswitch incl PCB incl met...,[],3pdt,2.333410e+11,[],7.99,{},Amplifiers & Effects,0.9136,Positive


In [57]:
instrument_data['instrument_category'].unique()

array(['Live Sound &amp', 'Band & Orchestra',
       'Wind & Woodwind Instrument', 'Microphones & Accessories',
       'Instrument Accessories', 'Keyboards', 'Live Sound & Stage', '',
       'Microphones &amp', 'Electronic Music',
       'Studio Recording Equipment', 'Drums &amp', 'Drums & Percussion',
       'Amplifiers & Effects', 'Guitars', 'Amplifiers &amp', 'Wind &amp',
       'Ukuleles', 'Stringed Instruments', 'Band &amp', 'Keyboards &amp'],
      dtype=object)

In [58]:
instrument_data['instrument_category'].value_counts()

Instrument Accessories        611517
Microphones & Accessories     123715
Live Sound & Stage            109311
Studio Recording Equipment     70574
Drums & Percussion             55956
Amplifiers & Effects           48236
                               37472
Electronic Music               35524
Wind & Woodwind Instrument     29199
Ukuleles                       27673
Band & Orchestra               26772
Guitars                        24216
Keyboards                      14620
Microphones &amp               12384
Live Sound &amp                10726
Drums &amp                      8295
Wind &amp                       5140
Band &amp                       3435
Amplifiers &amp                 3122
Stringed Instruments            1727
Keyboards &amp                   380
Name: instrument_category, dtype: int64

In [59]:
Instrument_Accessories = instrument_data[instrument_data['instrument_category']=='Instrument Accessories']
Microphones_Accessories = instrument_data[instrument_data['instrument_category']=='Microphones & Accessories']
Live_Sound_Stage = instrument_data[instrument_data['instrument_category']=='Live Sound & Stage']
Studio_Recording_Equipment = instrument_data[instrument_data['instrument_category']=='Studio Recording Equipment']
Drums_Percussion = instrument_data[instrument_data['instrument_category']=='Drums & Percussion']

In [60]:
print(Instrument_Accessories.shape)
print(Microphones_Accessories.shape)
print(Live_Sound_Stage.shape)
print(Studio_Recording_Equipment.shape)
print(Drums_Percussion.shape)

(611517, 22)
(123715, 22)
(109311, 22)
(70574, 22)
(55956, 22)


**Instrument_Accessories**

In [221]:
Instrument_Accessories_pos=Instrument_Accessories[Instrument_Accessories['vader_sentiment']=='Positive']
Instrument_Accessories_neg=Instrument_Accessories[Instrument_Accessories['vader_sentiment']=='Negative']

In [222]:
Instrument_Accessories_pos['asin'].value_counts()

B0002E1NNC    5288
B0002E1NWI    5288
B0002H0A3S    4280
B0002E3CK4    3492
B0002H03YY    3492
              ... 
B00A81LJ9W       1
B00N5IONY8       1
B00G446GR2       1
B00IABMIAQ       1
B0054RDWUE       1
Name: asin, Length: 22272, dtype: int64

In [223]:
Instrument_Accessories_pos[Instrument_Accessories_pos['asin']=='B0002E1NNC']

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,text_process,...,title,also_buy,brand,rank,also_view,price,details,instrument_category,compound,vader_sentiment
127608,5.0,False,"01 29, 2006",A1OYC08YGVJDBO,B0002E1NNC,NewForm Design,THE BEST - there's a good reason Elixirs are h...,Gwise,1138492800,best good reason elixir held top string matter...,...,Elixir Strings 80/20 Bronze Acoustic Guitar St...,[],Elixir,232010063.0,"['B000A6EUSC', 'B0002E1O7M', 'B0002E1O7W', 'B0...",12.55,{},Instrument Accessories,0.9022,Positive
127609,5.0,False,"01 29, 2006",A1OYC08YGVJDBO,B0002E1NNC,NewForm Design,THE BEST - there's a good reason Elixirs are h...,Gwise,1138492800,best good reason elixir held top string matter...,...,Elixir Strings 80/20 Bronze Acoustic Guitar St...,[],Elixir,232010063.0,"['B000A6EUSC', 'B0002E1O7M', 'B0002E1O7W', 'B0...",12.55,{},Instrument Accessories,0.9022,Positive
127610,5.0,False,"09 21, 2005",A3CH4VRNWB7Y03,B0002E1NNC,O. BLGN,I have a 25 year old Yamaha FG-200D(Domestic m...,No need for a thousand dollar guitar,1127260800,year old yamaha fg-d domestic model made japan...,...,Elixir Strings 80/20 Bronze Acoustic Guitar St...,[],Elixir,232010063.0,"['B000A6EUSC', 'B0002E1O7M', 'B0002E1O7W', 'B0...",12.55,{},Instrument Accessories,0.9565,Positive
127611,5.0,False,"09 21, 2005",A3CH4VRNWB7Y03,B0002E1NNC,O. BLGN,I have a 25 year old Yamaha FG-200D(Domestic m...,No need for a thousand dollar guitar,1127260800,year old yamaha fg-d domestic model made japan...,...,Elixir Strings 80/20 Bronze Acoustic Guitar St...,[],Elixir,232010063.0,"['B000A6EUSC', 'B0002E1O7M', 'B0002E1O7W', 'B0...",12.55,{},Instrument Accessories,0.9565,Positive
127612,4.0,False,"09 15, 2004",A2JV8PH4NGQPJ6,B0002E1NNC,Sir George Carlin Martin,I received a free sample of these strings in 1...,Great Tone - Long Life,1095206400,received free sample string believe hype reluc...,...,Elixir Strings 80/20 Bronze Acoustic Guitar St...,[],Elixir,232010063.0,"['B000A6EUSC', 'B0002E1O7M', 'B0002E1O7W', 'B0...",12.55,{},Instrument Accessories,0.8362,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134823,5.0,True,"05 13, 2018",A3TJRVEPTVC70O,B0002E1NNC,Robert D.,I've been playing guitar since 1976. I have us...,I've been playing guitar since 1976. I have us...,1526169600,playing guitar since used name brand string el...,...,Elixir Strings 80/20 Bronze Acoustic Guitar St...,[],Elixir,232010063.0,"['B000A6EUSC', 'B0002E1O7M', 'B0002E1O7W', 'B0...",12.55,{},Instrument Accessories,0.7783,Positive
134824,5.0,True,"05 13, 2018",A2JQGG0N9TKXWC,B0002E1NNC,cobraxrr,great product!,Five Stars,1526169600,great product,...,Elixir Strings 80/20 Bronze Acoustic Guitar St...,[],Elixir,232010063.0,"['B000A6EUSC', 'B0002E1O7M', 'B0002E1O7W', 'B0...",12.55,{},Instrument Accessories,0.6249,Positive
134825,5.0,True,"05 13, 2018",A2JQGG0N9TKXWC,B0002E1NNC,cobraxrr,great product!,Five Stars,1526169600,great product,...,Elixir Strings 80/20 Bronze Acoustic Guitar St...,[],Elixir,232010063.0,"['B000A6EUSC', 'B0002E1O7M', 'B0002E1O7W', 'B0...",12.55,{},Instrument Accessories,0.6249,Positive
134830,5.0,True,"05 11, 2018",A2OIS7QLY06TPU,B0002E1NNC,Robert Farley,Great price for a great string. Never liked co...,"Long-lasting, smooth tone",1525996800,great price great string never liked coated st...,...,Elixir Strings 80/20 Bronze Acoustic Guitar St...,[],Elixir,232010063.0,"['B000A6EUSC', 'B0002E1O7M', 'B0002E1O7W', 'B0...",12.55,{},Instrument Accessories,0.8412,Positive


In [224]:
Instrument_Accessories_neg['asin'].value_counts()

B0002E1NNC    130
B0002E1G5C    130
B0002E1NWI    130
B00KMFL70K     94
B00IFOTSJW     79
             ... 
B00KGHBMJA      1
B00XVMF93M      1
B004YXN23G      1
B00BCEKISM      1
B00U531UY2      1
Name: asin, Length: 3890, dtype: int64

In [225]:
Instrument_Accessories_neg[Instrument_Accessories_neg['asin']=='B0002E1G5C']

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,text_process,...,title,also_buy,brand,rank,also_view,price,details,instrument_category,compound,vader_sentiment
148480,2.0,True,"05 11, 2018",A15KRM4PXVM71A,B0002E1G5C,Amazon Customer,The first time using it wouldn't cut the B str...,Prepare to use wire cutters if you buy this.,1525996800,first time using cut b string high e string di...,...,DAddario Accessories Pro-Winder Guitar String ...,[],D'Addario Accessories,4210011.0,[],10.99,{},Instrument Accessories,-0.7269,Negative
148481,2.0,True,"05 11, 2018",A15KRM4PXVM71A,B0002E1G5C,Amazon Customer,The first time using it wouldn't cut the B str...,Prepare to use wire cutters if you buy this.,1525996800,first time using cut b string high e string di...,...,DAddario Accessories Pro-Winder Guitar String ...,[],D'Addario Accessories,4210011.0,[],10.99,{},Instrument Accessories,-0.7269,Negative
148528,2.0,True,"03 23, 2018",A3KZB16GDPJ7N4,B0002E1G5C,Scorps,It doesnt like to stay in place while winding ...,Nope,1521763200,doesnt like stay place winding string making f...,...,DAddario Accessories Pro-Winder Guitar String ...,[],D'Addario Accessories,4210011.0,[],10.99,{},Instrument Accessories,-0.7789,Negative
148529,2.0,True,"03 23, 2018",A3KZB16GDPJ7N4,B0002E1G5C,Scorps,It doesnt like to stay in place while winding ...,Nope,1521763200,doesnt like stay place winding string making f...,...,DAddario Accessories Pro-Winder Guitar String ...,[],D'Addario Accessories,4210011.0,[],10.99,{},Instrument Accessories,-0.7789,Negative
148542,2.0,True,"03 14, 2018",AEL2XKIZHCAMD,B0002E1G5C,Deron Lewis,"I like the winder and cutter, however the pin ...","Cutter and Winder, yes. Pin puller, no",1520985600,like winder cutter however pin puller feature ...,...,DAddario Accessories Pro-Winder Guitar String ...,[],D'Addario Accessories,4210011.0,[],10.99,{},Instrument Accessories,-0.5574,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153313,5.0,False,"11 25, 2010",A2T639IINN6BTG,B0002E1G5C,invisible,I bought one of these at my local music store ...,Good winder and pin puller but I don't use the...,1290643200,bought one local music store found sturdy easy...,...,DAddario Accessories Pro-Winder Guitar String ...,[],D'Addario Accessories,4210011.0,[],10.99,{},Instrument Accessories,-0.5719,Negative
153388,2.0,True,"05 15, 2018",ANF5WKA9RZHYI,B0002E1G5C,omi,"Works for the thicker strings (E/A), just bend...","Works for the thicker strings (E/A), just bend...",1526342400,work thicker string e bend string blade made w...,...,DAddario Accessories Pro-Winder Guitar String ...,[],D'Addario Accessories,4210011.0,[],10.99,{},Instrument Accessories,-0.7650,Negative
153389,2.0,True,"05 15, 2018",ANF5WKA9RZHYI,B0002E1G5C,omi,"Works for the thicker strings (E/A), just bend...","Works for the thicker strings (E/A), just bend...",1526342400,work thicker string e bend string blade made w...,...,DAddario Accessories Pro-Winder Guitar String ...,[],D'Addario Accessories,4210011.0,[],10.99,{},Instrument Accessories,-0.7650,Negative
153398,1.0,True,"05 13, 2018",A12MIA7YMSHLDR,B0002E1G5C,Amazon Customer,This one is not functional. The added cutter i...,The added cutter is so dull that it can't cut ...,1526169600,one functional added cutter dull cut string cl...,...,DAddario Accessories Pro-Winder Guitar String ...,[],D'Addario Accessories,4210011.0,[],10.99,{},Instrument Accessories,-0.8442,Negative


**Microphones_Accessories**

In [226]:
Microphones_Accessories_pos=Microphones_Accessories[Microphones_Accessories['vader_sentiment']=='Positive']
Microphones_Accessories_neg=Microphones_Accessories[Microphones_Accessories['vader_sentiment']=='Negative']

In [228]:
Microphones_Accessories_pos['asin'].value_counts()

B000XI6O2W    2072
B00DY1F2CS    2066
B00XQFONF4    1915
B0002GXF8Q    1582
B0002KZAKS    1470
              ... 
B00074V0DA       1
B000Z08TOE       1
B005O656H2       1
B00OZXS2EY       1
B00NLOP2IW       1
Name: asin, Length: 2897, dtype: int64

In [229]:
Microphones_Accessories_neg['asin'].value_counts()

B00DY1F2CS    168
B000XI6O2W    158
B00XQFONF4    125
B0002GXF8Q    102
B0002CZW0Y     76
             ... 
B0073K9B9G      1
B00AENI3L6      1
B001F4UN8Y      1
B003OUX91A      1
B0007XH9PY      1
Name: asin, Length: 872, dtype: int64

In [230]:
Microphones_Accessories_pos[Microphones_Accessories_pos['asin']=='B000XI6O2W']

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,text_process,...,title,also_buy,brand,rank,also_view,price,details,instrument_category,compound,vader_sentiment
472822,4.0,True,"11 10, 2008",A1RLQJHCKD4VXC,B000XI6O2W,Natzers,This is the perfect choice for a kids karaoke ...,Works Great!,1226275200,perfect choice kid karaoke machine work great ...,...,Singing Machine SMM-205 Unidirectional Dynamic...,[],Singing Machine,610013.0,[],0.91,{},Microphones & Accessories,0.8316,Positive
472825,5.0,True,"05 7, 2018",A1UCSDI6VBLWC3,B000XI6O2W,Joseph A.,Awesome,Five Stars,1525651200,awesome,...,Singing Machine SMM-205 Unidirectional Dynamic...,[],Singing Machine,610013.0,[],0.91,{},Microphones & Accessories,0.6249,Positive
472827,5.0,True,"05 3, 2018",A37K2SEY0JU1GP,B000XI6O2W,machatr,We wanted a second mic for my niece's Christma...,Legit,1525305600,wanted second mic niece christmas present work...,...,Singing Machine SMM-205 Unidirectional Dynamic...,[],Singing Machine,610013.0,[],0.91,{},Microphones & Accessories,0.7906,Positive
472832,4.0,True,"04 30, 2018",A2WGF1LP2U2FIM,B000XI6O2W,golflexus55,It's good to have. I liked,Four Stars,1525046400,good liked,...,Singing Machine SMM-205 Unidirectional Dynamic...,[],Singing Machine,610013.0,[],0.91,{},Microphones & Accessories,0.6908,Positive
472833,5.0,True,"04 29, 2018",A17BQ0QGTRGNNP,B000XI6O2W,Amazon Customer,Daughter has a karaoke machine. Bought this ad...,Great product.,1524960000,daughter karaoke machine bought additional mic...,...,Singing Machine SMM-205 Unidirectional Dynamic...,[],Singing Machine,610013.0,[],0.91,{},Microphones & Accessories,0.6369,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476719,5.0,True,"01 15, 2011",A3L68Q0QX3XTY3,B000XI6O2W,lftbhnd2,This microphone was a must have for my grandau...,Karoke Machine Must Have,1295049600,microphone must grandaughters karoke machine n...,...,Singing Machine SMM-205 Unidirectional Dynamic...,[],Singing Machine,610013.0,[],0.91,{},Microphones & Accessories,0.8225,Positive
476720,5.0,True,"05 23, 2018",A48U2OFPPXC7C,B000XI6O2W,Joseph Vaccaro,Worked great for my kids' karaoke machine,Five Stars,1527033600,worked great kid karaoke machine,...,Singing Machine SMM-205 Unidirectional Dynamic...,[],Singing Machine,610013.0,[],0.91,{},Microphones & Accessories,0.6249,Positive
476721,5.0,True,"05 22, 2018",AKDY3U5ETW00T,B000XI6O2W,Josephs,I got this to make announcements and toasts at...,Good quality,1526947200,got make announcement toast dad th party used ...,...,Singing Machine SMM-205 Unidirectional Dynamic...,[],Singing Machine,610013.0,[],0.91,{},Microphones & Accessories,0.7845,Positive
476728,4.0,True,"05 17, 2018",A2WUDTAL0ADE6S,B000XI6O2W,frankie frost,son loved it for his karaoke machine.,Four Stars,1526515200,son loved karaoke machine,...,Singing Machine SMM-205 Unidirectional Dynamic...,[],Singing Machine,610013.0,[],0.91,{},Microphones & Accessories,0.5994,Positive


In [231]:
Microphones_Accessories_neg[Microphones_Accessories_neg['asin']=='B00DY1F2CS']

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,text_process,...,title,also_buy,brand,rank,also_view,price,details,instrument_category,compound,vader_sentiment
822377,1.0,True,"06 18, 2018",A2SLR31FB4LFIG,B00DY1F2CS,Ash,This arm shouldn't be advertised for the Yeti....,Yeti Owners Beware!,1529280000,arm advertised yeti yeti heavy clamp support c...,...,NEEWER Adjustable Microphone Suspension Boom S...,[],Neewer,410013.0,[],13.5,{},Microphones & Accessories,-0.7783,Negative
822423,2.0,False,"05 27, 2018",A2VJDF9MV7IFEW,B00DY1F2CS,Margaryta,Lasted for a month before acting bad. Started ...,Lasted for a month before acting bad. Started ...,1527379200,lasted month acting bad started becoming loose...,...,NEEWER Adjustable Microphone Suspension Boom S...,[],Neewer,410013.0,[],13.5,{},Microphones & Accessories,-0.7003,Negative
822452,1.0,True,"05 16, 2018",A4OFX1X8SHWLV,B00DY1F2CS,Spodermen,Only bought this for the mic stand to tide me ...,It's a $30 mic kit that works just like a $30 ...,1526428800,bought mic stand tide couple week buy thing mi...,...,NEEWER Adjustable Microphone Suspension Boom S...,[],Neewer,410013.0,[],13.5,{},Microphones & Accessories,-0.6249,Negative
822453,1.0,True,"05 16, 2018",A16IK4ANTIE2TD,B00DY1F2CS,SovietCyborg,This mic boom arm is very low quality and brok...,Very weak and poor quality,1526428800,mic boom arm low quality broke weight micropho...,...,NEEWER Adjustable Microphone Suspension Boom S...,[],Neewer,410013.0,[],13.5,{},Microphones & Accessories,-0.6369,Negative
822464,1.0,True,"05 10, 2018",A9HVQP8IA6OZT,B00DY1F2CS,Colton Duffy,I just got mine. And trying to right the botto...,Now the whole thing is useless. Dont buy this,1525910400,got mine trying right bottom mount table screw...,...,NEEWER Adjustable Microphone Suspension Boom S...,[],Neewer,410013.0,[],13.5,{},Microphones & Accessories,-0.6597,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826063,3.0,True,"12 12, 2014",A1DM8MSZS8ADBG,B00DY1F2CS,Michelle Lee,Can't mount my AT2020 on to this. The place wh...,"No, kind of..",1418342400,mount at2020 place shockmount supposed mount c...,...,NEEWER Adjustable Microphone Suspension Boom S...,[],Neewer,410013.0,[],13.5,{},Microphones & Accessories,-0.5439,Negative
826073,1.0,True,"12 2, 2014",AXUJWU015P9FU,B00DY1F2CS,AndrewST,"This is one of the few products to make my ""Ha...","Weak, uninspired, and unusable.",1417478400,one product make hate list overall finish stan...,...,NEEWER Adjustable Microphone Suspension Boom S...,[],Neewer,410013.0,[],13.5,{},Microphones & Accessories,-0.9818,Negative
826116,1.0,True,"09 12, 2014",AAO1OWPLKPX2E,B00DY1F2CS,CJ Moretti,Dumb and Flimbsy,Flim Flam,1410480000,dumb flimbsy,...,NEEWER Adjustable Microphone Suspension Boom S...,[],Neewer,410013.0,[],13.5,{},Microphones & Accessories,-0.5106,Negative
826131,3.0,True,"08 21, 2014",A3UKMMTJPMQM4D,B00DY1F2CS,Ethan Dalool,"For $15, it does what it needs to do. The wors...",The worst part is the joint between the lower ...,1408579200,need worst part joint lower arm clamp quite bi...,...,NEEWER Adjustable Microphone Suspension Boom S...,[],Neewer,410013.0,[],13.5,{},Microphones & Accessories,-0.7884,Negative


**Live_Sound_Stage**

In [232]:
Live_Sound_Stage_pos=Live_Sound_Stage[Live_Sound_Stage['vader_sentiment']=='Positive']
Live_Sound_Stage_neg=Live_Sound_Stage[Live_Sound_Stage['vader_sentiment']=='Negative']

In [233]:
Live_Sound_Stage_pos['asin'].value_counts()

B000VJJQUU    1266
B000068NW8    1166
B0017H4EBG     974
B0002D0JX8     930
B00KUC2INU     896
              ... 
B00J5OBSTY       1
B00SXKENFM       1
B00DWKF7LW       1
B0172GCTVC       1
B0055U9MOK       1
Name: asin, Length: 4215, dtype: int64

In [234]:
Live_Sound_Stage_neg['asin'].value_counts()

B0002D0JX8    112
B000Z31G3M     99
B0017H4EBG     84
B000068O35     70
B0010K6TXQ     64
             ... 
B004B4D4TU      1
B00AEB5654      1
B006VR65ZA      1
B01BVPPOYY      1
B00LXDOV2Q      1
Name: asin, Length: 952, dtype: int64

In [235]:
Live_Sound_Stage_pos[Live_Sound_Stage_pos['asin']=='B000VJJQUU']

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,text_process,...,title,also_buy,brand,rank,also_view,price,details,instrument_category,compound,vader_sentiment
461259,5.0,True,"01 14, 2011",AGHFZZVB1F1FC,B000VJJQUU,Cashman&#039;s Comics,Great cable for not a lot of cash. If your ban...,Way cool cable for not a lot of cash,1294963200,great cable lot cash band like mine always goi...,...,CBI MLC LowZ XLR Male to XLR Female Microphone...,[],CBI Cables,134100444.0,[],9.95,{},Live Sound & Stage,0.6705,Positive
461261,5.0,True,"12 18, 2010",A19PZTLI008L8T,B000VJJQUU,David Henry,This is a great cable for a great price. High...,Great microphone cable for a great price,1292630400,great cable great price highly recommend using...,...,CBI MLC LowZ XLR Male to XLR Female Microphone...,[],CBI Cables,134100444.0,[],9.95,{},Live Sound & Stage,0.9755,Positive
461262,5.0,True,"11 1, 2010",A1UHAA9D70BCCI,B000VJJQUU,Amazon Customer,"Excellent product, low noise and cables came i...",Mic cable review,1288569600,excellent product low noise cable came individ...,...,CBI MLC LowZ XLR Male to XLR Female Microphone...,[],CBI Cables,134100444.0,[],9.95,{},Live Sound & Stage,0.6249,Positive
461263,4.0,True,"10 22, 2010",A1XJJ8H8EEP76,B000VJJQUU,James M. Laclair,Good value. Average quality. Not sure how lo...,Good value. Average quality,1287705600,good value average quality sure long last,...,CBI MLC LowZ XLR Male to XLR Female Microphone...,[],CBI Cables,134100444.0,[],9.95,{},Live Sound & Stage,0.7650,Positive
461265,2.0,True,"04 13, 2018",A3K9P6QCUU07US,B000VJJQUU,Christine Milazzo,This cable was super touchy to start with; how...,This cable was super touchy to start with,1523577600,cable super touchy start however used twice work,...,CBI MLC LowZ XLR Male to XLR Female Microphone...,[],CBI Cables,134100444.0,[],9.95,{},Live Sound & Stage,0.5994,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463876,5.0,True,"07 26, 2011",A357FXV24O4N9M,B000VJJQUU,Matt Sv (musical matt),"This chord, for the money is soooooo exxterrea...",The Best,1311638400,chord money soooooo exxterreamily durable long...,...,CBI MLC LowZ XLR Male to XLR Female Microphone...,[],CBI Cables,134100444.0,[],9.95,{},Live Sound & Stage,0.6908,Positive
463877,5.0,True,"07 13, 2011",A2O9W7LXRF2DGH,B000VJJQUU,Splotchy The Robot,If you are looking for some replacement or bac...,"A Good, Low Budget XLR",1310515200,looking replacement backup xlr cable great pri...,...,CBI MLC LowZ XLR Male to XLR Female Microphone...,[],CBI Cables,134100444.0,[],9.95,{},Live Sound & Stage,0.8479,Positive
463878,5.0,True,"05 22, 2018",A2UYGWMJQWHF41,B000VJJQUU,Craig Fisher,"Excellent product, glad I purchased.... Daught...",Worth it for sure,1526947200,excellent product glad purchased daughter happ...,...,CBI MLC LowZ XLR Male to XLR Female Microphone...,[],CBI Cables,134100444.0,[],9.95,{},Live Sound & Stage,0.8074,Positive
463883,5.0,True,"04 27, 2018",A13JAPZ2811TTN,B000VJJQUU,steve-o,Works good & good price,Good deal,1524787200,work good good price,...,CBI MLC LowZ XLR Male to XLR Female Microphone...,[],CBI Cables,134100444.0,[],9.95,{},Live Sound & Stage,0.7003,Positive


In [236]:
Live_Sound_Stage_neg[Live_Sound_Stage_neg['asin']=='B0002D0JX8']

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,text_process,...,title,also_buy,brand,rank,also_view,price,details,instrument_category,compound,vader_sentiment
107252,1.0,True,"03 28, 2018",A39AHPXCWB56C5,B0002D0JX8,Krisgirl,"Didn't work, just spit out little puffs of sm...","Didnt work, waste of money",1522195200,work spit little puff smoke died,...,Chauvet Lighting Chauvet DJ 700 Hurricane Fog ...,[],Chauvet,2.781100e+14,[],33.99,{},Live Sound & Stage,-0.5165,Negative
107253,1.0,True,"03 28, 2018",A39AHPXCWB56C5,B0002D0JX8,Krisgirl,"Didn't work, just spit out little puffs of sm...","Didnt work, waste of money",1522195200,work spit little puff smoke died,...,Chauvet Lighting Chauvet DJ 700 Hurricane Fog ...,[],Chauvet,2.781100e+14,[],33.99,{},Live Sound & Stage,-0.5165,Negative
107302,1.0,True,"12 27, 2017",A270GIAV8PBKNQ,B0002D0JX8,Alisha,I have a question if its used why is it filled...,I have a question if its used why is it ...,1514332800,question used filled black oil spread fog bott...,...,Chauvet Lighting Chauvet DJ 700 Hurricane Fog ...,[],Chauvet,2.781100e+14,[],33.99,{},Live Sound & Stage,-0.5423,Negative
107303,1.0,True,"12 27, 2017",A270GIAV8PBKNQ,B0002D0JX8,Alisha,I have a question if its used why is it filled...,I have a question if its used why is it ...,1514332800,question used filled black oil spread fog bott...,...,Chauvet Lighting Chauvet DJ 700 Hurricane Fog ...,[],Chauvet,2.781100e+14,[],33.99,{},Live Sound & Stage,-0.5423,Negative
107324,1.0,True,"11 30, 2017",A1ULJOUOER4BYE,B0002D0JX8,Anton,Terrible product. Used it occasionally. The he...,One Star,1512000000,terrible product used occasionally heater elem...,...,Chauvet Lighting Chauvet DJ 700 Hurricane Fog ...,[],Chauvet,2.781100e+14,[],33.99,{},Live Sound & Stage,-0.5770,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108937,1.0,False,"07 29, 2010",A2YYTMAV6RHYU9,B0002D0JX8,Guy One,I bought one of these foggers but did not take...,Beware poor workmanship,1280361600,bought one foggers take box year opportunity u...,...,Chauvet Lighting Chauvet DJ 700 Hurricane Fog ...,[],Chauvet,2.781100e+14,[],33.99,{},Live Sound & Stage,-0.8271,Negative
108956,1.0,True,"10 30, 2007",A2JQ69BEYUZ4CZ,B0002D0JX8,BeHappy,I purchased this Chauvet along with their fog ...,Yikes,1193702400,purchased chauvet along fog fluid want smell r...,...,Chauvet Lighting Chauvet DJ 700 Hurricane Fog ...,[],Chauvet,2.781100e+14,[],33.99,{},Live Sound & Stage,-0.5849,Negative
108957,1.0,True,"10 30, 2007",A2JQ69BEYUZ4CZ,B0002D0JX8,BeHappy,I purchased this Chauvet along with their fog ...,Yikes,1193702400,purchased chauvet along fog fluid want smell r...,...,Chauvet Lighting Chauvet DJ 700 Hurricane Fog ...,[],Chauvet,2.781100e+14,[],33.99,{},Live Sound & Stage,-0.5849,Negative
108982,1.0,True,"04 6, 2018",A1P6CEFNLRKH62,B0002D0JX8,Ray,Dead on arrival. You also have to manually tri...,Dead on arrival. You also have to manually tri...,1522972800,dead arrival also manually trigger using remote,...,Chauvet Lighting Chauvet DJ 700 Hurricane Fog ...,[],Chauvet,2.781100e+14,[],33.99,{},Live Sound & Stage,-0.6486,Negative


**Studio_Recording_Equipment**

In [237]:
Studio_Recording_Equipment_pos=Studio_Recording_Equipment[Studio_Recording_Equipment['vader_sentiment']=='Positive']
Studio_Recording_Equipment_neg=Studio_Recording_Equipment[Studio_Recording_Equipment['vader_sentiment']=='Negative']

In [238]:
Studio_Recording_Equipment_pos['asin'].value_counts()

B00KVEIY4E    1204
B003QKBVYK     922
B00IJ6QAO2     856
B001QWBM62     813
B000KGYAYQ     756
              ... 
B00EZA4URA       1
B000K6LEK4       1
B00B7OL1B0       1
B00HE8NCCO       1
B00KGH0QMY       1
Name: asin, Length: 2345, dtype: int64

In [239]:
Studio_Recording_Equipment_neg['asin'].value_counts()

B0027V760M    59
B0002E4YI8    58
B00IJ6QAO2    56
B00KVEIY4E    54
B00AK7SKL4    47
              ..
B00KVWK6M4     1
B00HI04QDC     1
B00186FIN4     1
B017KFVBGY     1
B01FL1J4NG     1
Name: asin, Length: 629, dtype: int64

In [240]:
Studio_Recording_Equipment_pos[Studio_Recording_Equipment_pos['asin']=='B00KVEIY4E']

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,text_process,...,title,also_buy,brand,rank,also_view,price,details,instrument_category,compound,vader_sentiment
917224,5.0,False,"08 23, 2014",A2T9EHCUD7CNEB,B00KVEIY4E,Karie Hoskins,We have a flat screen TV in our bedroom - and ...,Great Design & Sound,1408752000,flat screen tv bedroom sound predictably flat ...,...,"Mackie Studio Monitor, Black w/Green Trim, 3-i...",[],Mackie,5161002.0,[],14.99,{},Studio Recording Equipment,0.9856,Positive
917225,5.0,False,"08 22, 2014",A1DQHS7MOVYYYA,B00KVEIY4E,H.F.,These monitors are amazing! I usually lean tow...,Killer monitors!,1408665600,monitor amazing usually lean towards using rea...,...,"Mackie Studio Monitor, Black w/Green Trim, 3-i...",[],Mackie,5161002.0,[],14.99,{},Studio Recording Equipment,0.9809,Positive
917226,5.0,False,"08 20, 2014",AV44NCH7K65PC,B00KVEIY4E,William Carpenter,These speakers were easy to connect to my PC a...,These speakers were easy to connect to my PC a...,1408492800,speaker easy connect pc superb sound size good...,...,"Mackie Studio Monitor, Black w/Green Trim, 3-i...",[],Mackie,5161002.0,[],14.99,{},Studio Recording Equipment,0.9611,Positive
917227,5.0,False,"08 20, 2014",A1QOK9LU90ACF7,B00KVEIY4E,Centriculous,QUICK SUMMARY: These monitors provide a nice ...,Great pair of monitors for a home studio!,1408492800,quick summary monitor provide nice range frequ...,...,"Mackie Studio Monitor, Black w/Green Trim, 3-i...",[],Mackie,5161002.0,[],14.99,{},Studio Recording Equipment,0.9928,Positive
917228,5.0,False,"08 20, 2014",A2XH6KI5ZLRLPS,B00KVEIY4E,Tina (Redzworld),with the addition of a ten dollar converter......,like,1408492800,addition ten dollar converter use speaker tv l...,...,"Mackie Studio Monitor, Black w/Green Trim, 3-i...",[],Mackie,5161002.0,[],14.99,{},Studio Recording Equipment,0.9312,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918856,4.0,True,"08 5, 2018",A2D0BT2P9SQGOY,B00KVEIY4E,Dissolved,"The only studio monitors I've owned, but they ...",but they seem pretty good for the price,1533427200,studio monitor owned seem pretty good price pr...,...,"Mackie Studio Monitor, Black w/Green Trim, 3-i...",[],Mackie,5161002.0,[],14.99,{},Studio Recording Equipment,0.9716,Positive
918858,5.0,True,"08 4, 2018",A2G6FD27CB9U12,B00KVEIY4E,Nelson,Can't beat the price. Haven't had them at full...,but it sounds great!,1533340800,beat price full volume yet sound great,...,"Mackie Studio Monitor, Black w/Green Trim, 3-i...",[],Mackie,5161002.0,[],14.99,{},Studio Recording Equipment,0.6249,Positive
918859,1.0,True,"08 4, 2018",ADNL0HAZJP49K,B00KVEIY4E,Audiophile 1,These speakers are total garbage!!! Don't buy ...,These speakers are total garbage!!,1533340800,speaker total garbage buy reason use dynamic r...,...,"Mackie Studio Monitor, Black w/Green Trim, 3-i...",[],Mackie,5161002.0,[],14.99,{},Studio Recording Equipment,0.5233,Positive
918861,5.0,True,"07 31, 2018",A1C2N82G8JOZIN,B00KVEIY4E,krystal bermudez,Recomend 200%.Very surprised with the quality ...,Best! Very Cheap!,1532995200,recomend surprised quality sound love,...,"Mackie Studio Monitor, Black w/Green Trim, 3-i...",[],Mackie,5161002.0,[],14.99,{},Studio Recording Equipment,0.7269,Positive


In [241]:
Studio_Recording_Equipment_neg[Studio_Recording_Equipment_neg['asin']=='B0027V760M']

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,text_process,...,title,also_buy,brand,rank,also_view,price,details,instrument_category,compound,vader_sentiment
581897,2.0,True,"02 8, 2018",A3OFNR93YEZD6,B0027V760M,Chico Corrales,I had a total of 4 of these DI boxes. I recent...,I wasted quite a bit of time figuring that out,1518048000,total di box recently ordered two seem faulty ...,...,Premium Direct Injection Audio Box - Passive D...,[],Pyle,30051007.0,[],0.91,{},Studio Recording Equipment,-0.7650,Negative
581898,1.0,True,"02 1, 2018",A3L8Y22CYKEIMI,B0027V760M,thirty3na3rd,"We've just purchased (4) of these, two each fr...",Beware: Serious Defect!,1517443200,purchased two two different seller unusable ri...,...,Premium Direct Injection Audio Box - Passive D...,[],Pyle,30051007.0,[],0.91,{},Studio Recording Equipment,-0.9920,Negative
581902,1.0,True,"01 5, 2018",A22O66X6KMHJV1,B0027V760M,R&amp;B Gospel FAN,I bought two of these units and neither one is...,Not working out of the box,1515110400,bought two unit neither one giving output eith...,...,Premium Direct Injection Audio Box - Passive D...,[],Pyle,30051007.0,[],0.91,{},Studio Recording Equipment,-0.6293,Negative
581905,1.0,True,"12 30, 2017",A38ODT9LNS0VXR,B0027V760M,dapster,Piece of crap. It broke.,One Star,1514592000,piece crap broke,...,Premium Direct Injection Audio Box - Passive D...,[],Pyle,30051007.0,[],0.91,{},Studio Recording Equipment,-0.6597,Negative
581910,1.0,True,"11 26, 2017",A14ERK0O5QKM4B,B0027V760M,James A. Geiman,noise is so bad it is unusable,One Star,1511654400,noise bad unusable,...,Premium Direct Injection Audio Box - Passive D...,[],Pyle,30051007.0,[],0.91,{},Studio Recording Equipment,-0.5423,Negative
581913,1.0,False,"10 16, 2017",A3KRRBS1LKHVK0,B0027V760M,FotD,Bought one of these on eBay (new). Dead on ar...,It's a Paper Weight,1508112000,bought one ebay new dead arrival output kind x...,...,Premium Direct Injection Audio Box - Passive D...,[],Pyle,30051007.0,[],0.91,{},Studio Recording Equipment,-0.5719,Negative
581921,1.0,True,"09 4, 2017",ANZLPGA4DKXU0,B0027V760M,Shirley Meitzler,I bought 4. Out of the Box 2 were missing part...,Pyle... or Pile may be a better way to spell it.,1504483200,bought box missing part appeared water damage ...,...,Premium Direct Injection Audio Box - Passive D...,[],Pyle,30051007.0,[],0.91,{},Studio Recording Equipment,-0.6597,Negative
581924,1.0,False,"08 6, 2017",A1G6D4PJHPNG3C,B0027V760M,JamesR,"I'm not sure what these are supposed to be, I ...",Poorest quality! DON'T ORDER THIS ONE.,1501977600,sure supposed know thought purchasing direct b...,...,Premium Direct Injection Audio Box - Passive D...,[],Pyle,30051007.0,[],0.91,{},Studio Recording Equipment,-0.5719,Negative
581936,5.0,True,"05 24, 2017",A380413MZD4KZN,B0027V760M,Thomas C. Lang,Works as advertised. Price to performance rati...,Price to performance ratio can't be beaten,1495584000,work advertised price performance ratio beaten...,...,Premium Direct Injection Audio Box - Passive D...,[],Pyle,30051007.0,[],0.91,{},Studio Recording Equipment,-0.8928,Negative
581969,3.0,True,"01 2, 2017",A32UX0ZSZCA5WG,B0027V760M,John W.,"As with so many other reviews, one of the swit...","Very cheap in price and quality, but useful an...",1483315200,many review one switch bad took half hour open...,...,Premium Direct Injection Audio Box - Passive D...,[],Pyle,30051007.0,[],0.91,{},Studio Recording Equipment,-0.9149,Negative


**Drums_Percussion**

In [242]:
Drums_Percussion_pos=Drums_Percussion[Drums_Percussion['vader_sentiment']=='Positive']
Drums_Percussion_neg=Drums_Percussion[Drums_Percussion['vader_sentiment']=='Negative']

In [243]:
Drums_Percussion_pos['asin'].value_counts()

B0000775G0    1240
B0002F5CQK    1206
B00XEV4WYM     814
B0000775FZ     722
B01A6B0ICC     715
              ... 
B001VNUYJM       1
B016ARTCLY       1
B00PBBR2ZO       1
B01A9Q2OLM       1
B00XR3WRJE       1
Name: asin, Length: 3511, dtype: int64

In [244]:
Drums_Percussion_neg['asin'].value_counts()

B0002EJS9S    40
B0000775G0    20
B01FRN3N1C    20
B0002F4MKC    20
B004Z8ORXO    19
              ..
B0064CFJ0O     1
B00CMDOCNO     1
B019R8AJQK     1
B006RAEO0O     1
B01DKEDZFO     1
Name: asin, Length: 527, dtype: int64

In [245]:
Drums_Percussion_pos[Drums_Percussion_pos['asin']=='B0000775G0']

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,text_process,...,title,also_buy,brand,rank,also_view,price,details,instrument_category,compound,vader_sentiment
15500,5.0,False,"03 2, 2008",A1ZHIXGM9DD3I,B0000775G0,Gary Ng,I am a happy owner of this product. Hehe..\n\n...,My Meditation Chime,1204416000,happy owner product hehe pro beautifully craft...,...,"Woodstock Chimes Zenergy Chime, Solo, Silver","['B0000775FZ', '1782853278', 'B0000775FY', 'B0...",Woodstock Chimes,997100112.0,[],6.41,{},Drums & Percussion,0.9584,Positive
15501,5.0,False,"03 2, 2008",A1ZHIXGM9DD3I,B0000775G0,Gary Ng,I am a happy owner of this product. Hehe..\n\n...,My Meditation Chime,1204416000,happy owner product hehe pro beautifully craft...,...,"Woodstock Chimes Zenergy Chime, Solo, Silver","['B0000775FZ', '1782853278', 'B0000775FY', 'B0...",Woodstock Chimes,997100112.0,[],6.41,{},Drums & Percussion,0.9584,Positive
15504,5.0,True,"02 6, 2007",A2JOVHUZO1KDFE,B0000775G0,E. Maya,"Makes beautiful sound, perfect to end a medita...",Beautiful sound,1170720000,make beautiful sound perfect end meditation se...,...,"Woodstock Chimes Zenergy Chime, Solo, Silver","['B0000775FZ', '1782853278', 'B0000775FY', 'B0...",Woodstock Chimes,997100112.0,[],6.41,{},Drums & Percussion,0.9136,Positive
15505,5.0,True,"02 6, 2007",A2JOVHUZO1KDFE,B0000775G0,E. Maya,"Makes beautiful sound, perfect to end a medita...",Beautiful sound,1170720000,make beautiful sound perfect end meditation se...,...,"Woodstock Chimes Zenergy Chime, Solo, Silver","['B0000775FZ', '1782853278', 'B0000775FY', 'B0...",Woodstock Chimes,997100112.0,[],6.41,{},Drums & Percussion,0.9136,Positive
15506,5.0,False,"01 31, 2007",A1OUH3MNAT11ZD,B0000775G0,Raphael W. Stephens III,The Zenergy Solo Chime has a wonderfully beaut...,Zenfully beautiful,1170201600,zenergy solo chime wonderfully beautiful reson...,...,"Woodstock Chimes Zenergy Chime, Solo, Silver","['B0000775FZ', '1782853278', 'B0000775FY', 'B0...",Woodstock Chimes,997100112.0,[],6.41,{},Drums & Percussion,0.9531,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17133,4.0,True,"05 2, 2018",A1MDEDVU2KO74U,B0000775G0,jim guerin,Wonderful little chime to grab students' atten...,Wonderful little chime to grab students' atten...,1525219200,wonderful little chime grab student attention ...,...,"Woodstock Chimes Zenergy Chime, Solo, Silver","['B0000775FZ', '1782853278', 'B0000775FY', 'B0...",Woodstock Chimes,997100112.0,[],6.41,{},Drums & Percussion,0.5859,Positive
17136,5.0,True,"04 28, 2018",A2QX0N9DN1V1NI,B0000775G0,P. Wingenfeld,Well made product. Nice tone.,Nice tone.,1524873600,well made product nice tone,...,"Woodstock Chimes Zenergy Chime, Solo, Silver","['B0000775FZ', '1782853278', 'B0000775FY', 'B0...",Woodstock Chimes,997100112.0,[],6.41,{},Drums & Percussion,0.5994,Positive
17137,5.0,True,"04 28, 2018",A2QX0N9DN1V1NI,B0000775G0,P. Wingenfeld,Well made product. Nice tone.,Nice tone.,1524873600,well made product nice tone,...,"Woodstock Chimes Zenergy Chime, Solo, Silver","['B0000775FZ', '1782853278', 'B0000775FY', 'B0...",Woodstock Chimes,997100112.0,[],6.41,{},Drums & Percussion,0.5994,Positive
17138,5.0,True,"04 25, 2018",A30ZOA7FQ0HX7O,B0000775G0,ZM,Great purchase!,Five Stars,1524614400,great purchase,...,"Woodstock Chimes Zenergy Chime, Solo, Silver","['B0000775FZ', '1782853278', 'B0000775FY', 'B0...",Woodstock Chimes,997100112.0,[],6.41,{},Drums & Percussion,0.6249,Positive


In [246]:
Drums_Percussion_neg[Drums_Percussion_neg['asin']=='B0002EJS9S']

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,text_process,...,title,also_buy,brand,rank,also_view,price,details,instrument_category,compound,vader_sentiment
193364,1.0,True,"10 16, 2016",A22RVJ8BSQQ8OC,B0002EJS9S,Bobby Fetta,The part that you hit isn't nearly tight enoug...,Waste of money.,1476576000,part hit nearly tight enough lose bubbly almos...,...,Rhythm Band Bongos Junior 6 in. H x 5 in. and ...,[],Rhythm Band,3.100810e+12,"['B006GFFPLM', 'B007SPR25G', 'B001RY82YE', 'B0...",7.99,{},Drums & Percussion,-0.6728,Negative
193365,1.0,True,"10 16, 2016",A22RVJ8BSQQ8OC,B0002EJS9S,Bobby Fetta,The part that you hit isn't nearly tight enoug...,Waste of money.,1476576000,part hit nearly tight enough lose bubbly almos...,...,Rhythm Band Bongos Junior 6 in. H x 5 in. and ...,[],Rhythm Band,3.100810e+12,"['B006GFFPLM', 'B007SPR25G', 'B001RY82YE', 'B0...",7.99,{},Drums & Percussion,-0.6728,Negative
193374,1.0,True,"09 16, 2016",A3ES8LIPLRKD3F,B0002EJS9S,Kelsey and Jackie,HORRIBLE DO NOT BUY I'm extremely disappointed...,HORRIBLE would give 0 stars if possible,1473984000,horrible buy extremely disappointed bought yea...,...,Rhythm Band Bongos Junior 6 in. H x 5 in. and ...,[],Rhythm Band,3.100810e+12,"['B006GFFPLM', 'B007SPR25G', 'B001RY82YE', 'B0...",7.99,{},Drums & Percussion,-0.8265,Negative
193375,1.0,True,"09 16, 2016",A3ES8LIPLRKD3F,B0002EJS9S,Kelsey and Jackie,HORRIBLE DO NOT BUY I'm extremely disappointed...,HORRIBLE would give 0 stars if possible,1473984000,horrible buy extremely disappointed bought yea...,...,Rhythm Band Bongos Junior 6 in. H x 5 in. and ...,[],Rhythm Band,3.100810e+12,"['B006GFFPLM', 'B007SPR25G', 'B001RY82YE', 'B0...",7.99,{},Drums & Percussion,-0.8265,Negative
193400,1.0,True,"06 16, 2016",AV02G42ACWWP9,B0002EJS9S,Ryan Mcbride,I bought these for my kids so we could have a ...,"Total Garbage. Seriously, don't buy.",1466035200,bought kid could jam session quality awful ski...,...,Rhythm Band Bongos Junior 6 in. H x 5 in. and ...,[],Rhythm Band,3.100810e+12,"['B006GFFPLM', 'B007SPR25G', 'B001RY82YE', 'B0...",7.99,{},Drums & Percussion,-0.6908,Negative
193401,1.0,True,"06 16, 2016",AV02G42ACWWP9,B0002EJS9S,Ryan Mcbride,I bought these for my kids so we could have a ...,"Total Garbage. Seriously, don't buy.",1466035200,bought kid could jam session quality awful ski...,...,Rhythm Band Bongos Junior 6 in. H x 5 in. and ...,[],Rhythm Band,3.100810e+12,"['B006GFFPLM', 'B007SPR25G', 'B001RY82YE', 'B0...",7.99,{},Drums & Percussion,-0.6908,Negative
193406,1.0,True,"05 10, 2016",A1S71OGK5VZHHE,B0002EJS9S,Mom of two,I would give this zero stars if I could! Terr...,Would NOT recommend!,1462838400,would give zero star could terribly cheap pape...,...,Rhythm Band Bongos Junior 6 in. H x 5 in. and ...,[],Rhythm Band,3.100810e+12,"['B006GFFPLM', 'B007SPR25G', 'B001RY82YE', 'B0...",7.99,{},Drums & Percussion,-0.5574,Negative
193407,1.0,True,"05 10, 2016",A1S71OGK5VZHHE,B0002EJS9S,Mom of two,I would give this zero stars if I could! Terr...,Would NOT recommend!,1462838400,would give zero star could terribly cheap pape...,...,Rhythm Band Bongos Junior 6 in. H x 5 in. and ...,[],Rhythm Band,3.100810e+12,"['B006GFFPLM', 'B007SPR25G', 'B001RY82YE', 'B0...",7.99,{},Drums & Percussion,-0.5574,Negative
193424,1.0,False,"02 19, 2016",A2DPRNNDPZOIJZ,B0002EJS9S,Repairman123,bad bad drum. broke after 4 uses. the top sh...,One Star,1455840000,bad bad drum broke us top sheet cracked bad ba...,...,Rhythm Band Bongos Junior 6 in. H x 5 in. and ...,[],Rhythm Band,3.100810e+12,"['B006GFFPLM', 'B007SPR25G', 'B001RY82YE', 'B0...",7.99,{},Drums & Percussion,-0.9432,Negative
193425,1.0,False,"02 19, 2016",A2DPRNNDPZOIJZ,B0002EJS9S,Repairman123,bad bad drum. broke after 4 uses. the top sh...,One Star,1455840000,bad bad drum broke us top sheet cracked bad ba...,...,Rhythm Band Bongos Junior 6 in. H x 5 in. and ...,[],Rhythm Band,3.100810e+12,"['B006GFFPLM', 'B007SPR25G', 'B001RY82YE', 'B0...",7.99,{},Drum

In [87]:
!pip install -U scikit-learn

In [88]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import seaborn as sns
#from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
import gzip

In [89]:
instrument_data.shape

(1259994, 22)

In [90]:
counts1=instrument_data['reviewerID'].value_counts() 
counts=instrument_data['asin'].value_counts()

In [91]:
counts

B0002E1NWI    7224
B0002E1NNC    7224
B0002H0A3S    6192
B0002E3CK4    5014
B0002H03YY    5014
              ... 
B005HZ5K12       1
B00JB6P3EM       1
B00DK439G0       1
B004U5D7BU       1
B007SRBIYU       1
Name: asin, Length: 53699, dtype: int64

In [92]:
counts1

A2NYK9KWFMJV4Y    481
A3YVD62FSVIUJ     244
A396ELNTQDFYPO    219
ADH0O8UVJOT10     154
A15TYOEWBQYF0X    149
                 ... 
A2AL3YI00YTPT0      1
A32J5NQH4MQDT1      1
A3CUCLH0ANA4FX      1
A1ELYDHCB392A1      1
AZMQ27V1ZKWHP       1
Name: reviewerID, Length: 663691, dtype: int64

In [93]:
len(counts1[counts1>=50].index)

164

In [94]:
len(counts[counts>=50].index)

4312

In [172]:
df1=instrument_data[instrument_data['reviewerID'].isin(counts1[counts1 >=50].index)]
df1.shape

(11787, 22)

In [173]:
df1=df1[df1['asin'].isin(counts[counts >=50].index)]
df1.shape

(7738, 22)

In [174]:
df1.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,text_process,...,title,also_buy,brand,rank,also_view,price,details,instrument_category,compound,vader_sentiment
50,5.0,True,"12 26, 2016",A1VPJT3BHZ9UAS,9792372326,Tech Geek Reviews,I use with Nord Piano 2 or Roland RD-300NX for...,"Fantastic as wedge monitor, powerfully enough ...",1482710400,use nord piano roland rd-nx mostly piano organ...,...,"QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",[],QSC,6.669710e+15,"['B06Y4BT184', 'B00CE0VLXO', 'B00NIE80A2', 'B0...",125.99,{},Live Sound &amp,0.9657,Positive
704,4.0,True,"01 2, 2018",A1H2D403IZ5E6K,B00004TT3S,Dr H,Putting these caps on your boom-whackers essen...,"Exactly what they're supposed to be, but overp...",1514851200,putting cap boom-whackers essentially turn sta...,...,Boomwhackers Octavator Tube Caps,[],Boomwhackers,4.941100e+09,[],9.99,{},Instrument Accessories,0.9186,Positive
705,4.0,True,"01 2, 2018",A1H2D403IZ5E6K,B00004TT3S,Dr H,Putting these caps on your boom-whackers essen...,"Exactly what they're supposed to be, but overp...",1514851200,putting cap boom-whackers essentially turn sta...,...,Boomwhackers Octavator Tube Caps,[],Boomwhackers,4.941100e+09,[],9.99,{},Instrument Accessories,0.9186,Positive
1010,5.0,True,"08 16, 2017",ATJN91PH28F6T,B00005ML71,Randall C Kemp,"For what you pay, this is a great pedal. (Got ...",Works Great with my Yamaha Reface,1502841600,pay great pedal got discount prime expecting r...,...,Yamaha FC5 Compact Sustain Pedal for Portable ...,[],Yamaha,4.101002e+07,[],14.35,{},Instrument Accessories,0.9136,Positive
1011,5.0,True,"08 16, 2017",ATJN91PH28F6T,B00005ML71,Randall C Kemp,"For what you pay, this is a great pedal. (Got ...",Works Great with my Yamaha Reface,1502841600,pay great pedal got discount prime expecting r...,...,Yamaha FC5 Compact Sustain Pedal for Portable ...,[],Yamaha,4.101002e+07,[],14.35,{},Instrument Accessories,0.9136,Positive


In [175]:
ratings_sum = pd.DataFrame(df1.groupby(['asin'])['overall'].sum()).rename(columns = {'overall': 'ratings_sum'})
top10 = ratings_sum.sort_values('ratings_sum', ascending = False).head(10)
top10

,ratings_sum
asin,
B0002H03YY,646.0
B0002E3CK4,646.0
B0002H05BA,646.0
B0002H0A3S,430.0
B0006LOBA8,323.0
B0002E1G5C,274.0
B0002E1J3Q,264.0
B0007Y09VO,215.0
B0002E1NWI,206.0


In [176]:
top10_popular=top10.merge(df1,left_index = True, right_on = 'asin').drop_duplicates(
    ['asin', 'title'])[['asin', 'title','ratings_sum']]

In [177]:
print ('Top 10 Popular Products by sum user ratings\n')
top10_popular

Top 10 Popular Products by sum user ratings



,asin,title,ratings_sum
271400,B0002H03YY,D'Addario EXL120+ Nickel Wound Electric Guitar...,646.0
174224,B0002E3CK4,D'Addario EXL120-7 Nickel Wound 7-String Elect...,646.0
289752,B0002H05BA,D'Addario EJ21 Nickel Wound Electric Guitar St...,646.0
282252,B0002H0A3S,D'Addario EJ26 Phosphor Bronze Acoustic Guitar...,430.0
346618,B0006LOBA8,D'Addario EXL150H Nickel Wound Electric Guitar...,323.0
148466,B0002E1G5C,DAddario Accessories Pro-Winder Guitar String ...,274.0
122498,B0002E1J3Q,"Ernie Ball Not Even Slinky Nickel Wound Set, ....",264.0
365126,B0007Y09VO,D'Addario EJ17-B25 Phosphor Bronze Acoustic Gu...,215.0
139740,B0002E1NWI,Elixir Strings 80/20 Bronze Acoustic Guitar St...,206.0
128188,B0002E1NNC,Elixir Strings 80/20 Bronze Acoustic Guitar St...,206.0


In [178]:
df1.drop_duplicates(subset='asin',inplace=True)

In [179]:
ratingsd=df1.pivot(index='asin',columns= 'reviewerID',values='overall').fillna(0)
ratingsd.head()

reviewerID,A0096681Y127OL1H8W3U,A10APIDAZISWQF,A10E3QH2FQUBLF,A129UKDJUW5VSZ,A129WUT5OMBKG,A134J0S1HWE4F7,A136IQFGB01KQB,A15TYOEWBQYF0X,A15UON6342H4YU,A16FNRY0AK3MUL,...,ATJN91PH28F6T,ATPP3IWN4S9PW,ATZCRPINM5F87,AWY308ST6QBNK,AXWEQHTXQWR7Q,AYP7YOSLPC5NP,AZAYBFPLEDFL7,AZBUUKQLYKUCL,AZV2U6GU5QA6C,AZVME8JMPD3F4
asin,,,,,,,,,,,,,,,,,,,,,
9792372326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00004TT3S,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00005ML71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B000068NSX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B000068NTU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [180]:
ratingsd.shape

(2058, 164)

In [181]:
from sklearn.model_selection import train_test_split
traind, testd = train_test_split(ratingsd, test_size=0.30,random_state=42)

In [182]:
train = traind.values
test = testd.values

In [183]:
sparsity = float(len(train.nonzero()[0]))
sparsity /= (train.shape[0] * train.shape[1])
sparsity *= 100
print ('Sparsity: {:5.2f}%'.format(sparsity))

Sparsity:  0.61%


In [184]:
def item_similarity(ratings, epsilon=1e-9):
    # epsilon -> for handling dived-by-zero errors
    sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

In [185]:
item_sim = item_similarity(train)

In [186]:
def predict_item(ratings, similarity):
    return ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

In [187]:
item_prediction = predict_item(train, item_sim)

In [188]:
item_prediction[:4, :4]

array([[1.43643162e-11, 1.73532483e-11, 1.80364603e-11, 1.49278340e-11],
       [2.40562094e-11, 2.90618341e-11, 3.02060230e-11, 2.49999441e-11],
       [1.22363657e-11, 1.47825130e-11, 1.53645130e-11, 1.27164032e-11],
       [1.82573793e-11, 2.20563813e-11, 2.29247596e-11, 1.89736235e-11]])

In [189]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [190]:
print ('Item-based CF MSE: ' + str(get_mse(item_prediction, test)))

Item-based CF MSE: 20.131067256822405


In [191]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
r_mat_tr=svd.fit_transform(traind) 
print(svd.explained_variance_ratio_)  
print(svd.explained_variance_ratio_.sum())

#pm=pd.DataFrame(cosine_similarity(r_mat_tr))
#pm.head()
ctrain = cosine_similarity(r_mat_tr)

[0.02554427 0.01830277 0.01801774 0.01718369 0.01546388 0.01501689
 0.01483546 0.01471673 0.01336688 0.01276534 0.01210831 0.01189015
 0.01170332 0.01138278 0.01104324 0.01100498 0.01100097 0.01075125
 0.01094797 0.01088696 0.01094875 0.01018326 0.01003621 0.00980696
 0.00940352 0.00922366 0.00906385 0.00907697 0.00889271 0.00900819
 0.00888038 0.00837524 0.00833286 0.0083816  0.00827823 0.00809817
 0.00805285 0.00812612 0.00809213 0.00798074 0.00782902 0.0075538
 0.00739231 0.00715571 0.00734155 0.00719964 0.00724169 0.00721116
 0.00713311 0.00692744]
0.5291614291301067


In [192]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
r_mat_tr=svd.fit_transform(testd) 
print(svd.explained_variance_ratio_)  
print(svd.explained_variance_ratio_.sum())

#pmtt=pd.DataFrame(cosine_similarity(r_mat_tr))
#print (pmtt[:2])
#pmtt.head()
ctest = cosine_similarity(r_mat_tr)


[0.03399278 0.02801936 0.0210731  0.019837   0.01940283 0.01884532
 0.01841084 0.01709485 0.01576887 0.01605337 0.01573888 0.01518472
 0.01395524 0.01323648 0.01195146 0.01204746 0.0118069  0.01209728
 0.01113537 0.01136089 0.01066337 0.01053559 0.01008588 0.01004529
 0.01007948 0.0100675  0.00984211 0.00941548 0.00936171 0.00926985
 0.00927873 0.00884317 0.00862426 0.00841554 0.0085598  0.00859652
 0.00805284 0.00798536 0.0079093  0.0080104  0.00791604 0.0077567
 0.00741297 0.00730556 0.00727906 0.0072899  0.00716589 0.00725578
 0.00711315 0.00715674]
0.5943069924565809


In [193]:
print (' CF MSE: ' + str(get_mse(ctrain, ctest)))

 CF MSE: 0.07175336025371977


In [194]:
df1 = df1.sort_values(by='overall')
df1 = df1.reset_index(drop=True)
count_users = df1.groupby("reviewerID", as_index=False).count()

In [195]:
count_users

,reviewerID,overall,verified,reviewTime,asin,reviewerName,reviewText,summary,unixReviewTime,text_process,...,title,also_buy,brand,rank,also_view,price,details,instrument_category,compound,vader_sentiment
0,A0096681Y127OL1H8W3U,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15
1,A10APIDAZISWQF,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
2,A10E3QH2FQUBLF,13,13,13,13,13,13,13,13,13,...,13,13,13,13,13,13,13,13,13,13
3,A129UKDJUW5VSZ,12,12,12,12,12,12,12,12,12,...,12,12,12,12,12,12,12,12,12,12
4,A129WUT5OMBKG,17,17,17,17,17,17,17,17,17,...,17,17,17,17,17,17,17,17,17,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,AYP7YOSLPC5NP,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
160,AZAYBFPLEDFL7,17,17,17,17,17,17,17,17,17,...,17,17,17,17,17,17,17,17,17,17
161,AZBUUKQLYKUCL,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
162,AZV2U6GU5QA6C,23,23,23,23,23,23,23,23,23,...,23,23,23,23,23,23,23,23,23,23


In [196]:
count = df1.groupby("asin", as_index=False).mean()

In [197]:
items_df = count[['asin']]
items_df.head()
print(len(items_df))

2058


In [198]:
users_df = count_users[['reviewerID']]
users_df.head()
print(len(users_df))

164


In [199]:
users_list = users_df.values
len(users_list)

164

In [200]:
df_clean_matrix = df1.pivot(index='asin', columns='reviewerID', values='overall').fillna(0)
df_clean_matrix = df_clean_matrix.T
R = (df_clean_matrix).values
R

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [201]:
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)
R_demeaned

array([[-0.03498542, -0.03498542, -0.03498542, ..., -0.03498542,
        -0.03498542, -0.03498542],
       [-0.01652089, -0.01652089, -0.01652089, ..., -0.01652089,
        -0.01652089, -0.01652089],
       [-0.02623907, -0.02623907, -0.02623907, ..., -0.02623907,
        -0.02623907, -0.02623907],
       ...,
       [-0.0223518 , -0.0223518 , -0.0223518 , ..., -0.0223518 ,
        -0.0223518 , -0.0223518 ],
       [-0.05199223, -0.05199223, -0.05199223, ..., -0.05199223,
        -0.05199223, -0.05199223],
       [-0.01166181, -0.01166181, -0.01166181, ..., -0.01166181,
        -0.01166181, -0.01166181]])

In [202]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned)

In [203]:
sigma = np.diag(sigma)

In [204]:
users_df

,reviewerID
0,A0096681Y127OL1H8W3U
1,A10APIDAZISWQF
2,A10E3QH2FQUBLF
3,A129UKDJUW5VSZ
4,A129WUT5OMBKG
...,...
159,AYP7YOSLPC5NP
160,AZAYBFPLEDFL7
161,AZBUUKQLYKUCL
162,AZV2U6GU5QA6C


In [205]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = df_clean_matrix.columns)
preds_df['reviewerID'] = users_df
preds_df.set_index('reviewerID', inplace=True)
preds_df.head()

asin,9792372326,B00004TT3S,B00005ML71,B000068NSX,B000068NTU,B000068NUO,B000068NUQ,B000068NVJ,B000068NW8,B000068NW9,...,B01FHM1DNS,B01FRN3N1C,B01G3JJ7B4,B01GHGJ8AI,B01GHUA3XA,B01GMW448S,B01GPXSGX8,B01GTOO9IO,B01H31ITWE,B01HCNLHJA
reviewerID,,,,,,,,,,,,,,,,,,,,,
A0096681Y127OL1H8W3U,0.039533,0.038460,0.038553,0.037928,0.004856,0.038978,0.038978,0.037848,0.039870,0.038505,...,0.039853,0.037870,0.038800,0.038605,0.039149,0.037983,0.038258,0.038202,-0.000710,0.000585
A10APIDAZISWQF,0.017984,0.017649,0.017680,0.017482,0.008589,0.017811,0.017811,0.017457,0.018088,0.017664,...,0.018083,0.017463,0.017756,0.017696,0.017864,0.017500,0.017586,0.017569,0.005229,0.006552
A10E3QH2FQUBLF,0.028885,0.028274,0.028329,0.027970,0.010971,0.028570,0.028570,0.027923,0.029076,0.028301,...,0.029066,0.027935,0.028469,0.028358,0.028666,0.028002,0.028159,0.028127,0.005651,0.007577
A129UKDJUW5VSZ,0.030762,0.030049,0.030113,0.029694,0.009140,0.030394,0.030394,0.029640,0.030986,0.030080,...,0.030974,0.029655,0.030276,0.030147,0.030507,0.029731,0.029915,0.029877,0.003740,0.005523
A129WUT5OMBKG,0.045856,0.044417,0.044539,0.043708,-0.003821,0.045109,0.045109,0.043600,0.046310,0.044476,...,0.046286,0.043631,0.044870,0.044609,0.045340,0.043779,0.044145,0.044071,-0.007299,-0.007553


In [206]:
preds_df.shape

(164, 2058)

In [207]:
def recommend_it(predictions_df, itm_df, original_ratings_df, num_recommendations=10,ruserId='A0096681Y127OL1H8W3U'):
    
    # Get and sort the user's predictions
    sorted_user_predictions = predictions_df.loc[ruserId].sort_values(ascending=False)
    
    # Get the user's data and merge in the item information.
    user_data = original_ratings_df[original_ratings_df.reviewerID == ruserId]
    user_full = (user_data.merge(itm_df, how = 'left', left_on = 'asin', right_on = 'asin').
                     sort_values(['overall'], ascending=False)
                 )

    print ('User {0} has already purchased {1} items.'.format(ruserId, user_full.shape[0]))
    print ('Recommending the highest {0} predicted  items not already purchased.'.format(num_recommendations))
    
    # Recommend the highest predicted rating items that the user hasn't bought yet.
    recommendations = (itm_df[~itm_df['asin'].isin(user_full['asin'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'asin',
               right_on = 'asin').
         rename(columns = {ruserId: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )
    topk=recommendations.merge(original_ratings_df,left_index = True, right_on = 'asin',left_on='asin').drop_duplicates(
    ['asin', 'title'])[['asin', 'title']]

    return topk


In [209]:
recommend_it(preds_df, items_df, df1, 5,'A129WUT5OMBKG')

User A129WUT5OMBKG has already purchased 17 items.
Recommending the highest 5 predicted  items not already purchased.


,asin,title
778,B0055V7UR0,"Dunlop PVP101 Pick Variety Pack, Assorted, Lig..."
1801,B001SBN6BA,Your Cable Store 6 Foot XLR 3 Pin Female To 1/...
1977,B000RKVH0K,GLS Audio Vocal Microphone ES-58-S &amp; Mic C...
1961,B000RW1NVG,Boss NS-2 Noise Suppressor Pedal
1100,B00AHEWBM4,Xotic SP Compressor Pedal


In [210]:
######################################################################

In [211]:
preds_df.shape

(164, 2058)

In [212]:
X = preds_df.T
X.head()

reviewerID,A0096681Y127OL1H8W3U,A10APIDAZISWQF,A10E3QH2FQUBLF,A129UKDJUW5VSZ,A129WUT5OMBKG,A134J0S1HWE4F7,A136IQFGB01KQB,A15TYOEWBQYF0X,A15UON6342H4YU,A16FNRY0AK3MUL,...,ATJN91PH28F6T,ATPP3IWN4S9PW,ATZCRPINM5F87,AWY308ST6QBNK,AXWEQHTXQWR7Q,AYP7YOSLPC5NP,AZAYBFPLEDFL7,AZBUUKQLYKUCL,AZV2U6GU5QA6C,AZVME8JMPD3F4
asin,,,,,,,,,,,,,,,,,,,,,
9792372326,0.039533,0.017984,0.028885,0.030762,0.045856,0.012626,0.009425,0.029092,0.039323,0.013168,...,0.034506,0.031947,0.022753,0.030101,0.048363,0.025128,0.040521,0.024558,0.063272,0.012623
B00004TT3S,0.038460,0.017649,0.028274,0.030049,0.044417,0.012407,0.009271,0.028429,0.038304,0.012936,...,0.033714,0.031182,0.022328,0.029431,0.046782,0.024600,0.039446,0.024050,0.060465,0.012404
B00005ML71,0.038553,0.017680,0.028329,0.030113,0.044539,0.012427,0.009285,0.028488,0.038393,0.012957,...,0.033784,0.031250,0.022367,0.029491,0.046916,0.024648,0.039540,0.024096,0.060689,0.012425
B000068NSX,0.037928,0.017482,0.027970,0.029694,0.043708,0.012297,0.009194,0.028099,0.037799,0.012819,...,0.033320,0.030802,0.022115,0.029097,0.046005,0.024336,0.038914,0.023796,0.059100,0.012295
B000068NTU,0.004856,0.008589,0.010971,0.009140,-0.003821,0.006591,0.005273,0.009133,0.007025,0.006743,...,0.010578,0.008470,0.010794,0.010087,-0.007303,0.009655,0.006145,0.009754,-0.054097,0.006612


In [213]:
X1 = X

In [214]:
#Decomposing the Matrix
from sklearn.decomposition import TruncatedSVD
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(2058, 10)

In [215]:
#Correlation Matrix

correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(2058, 2058)

In [216]:
X.index[75]

'B0002D00F0'

In [217]:
i = "B0002D00F0"

product_names = list(X.index)
product_ID = product_names.index(i)
product_ID

75

In [218]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

(2058,)

In [220]:
Recommend = list(X.index[correlation_product_ID > 0.65])

# Removes the item already bought by the customer
Recommend.remove(i) 

Recommend[0:5]

['9792372326', 'B00004TT3S', 'B00005ML71', 'B000068NSX', 'B000068NUO']